In [56]:
import pandas as pd
import numpy as np
import sklearn
import os
import matplotlib.pyplot as plt
import time

In [57]:
# 加载数据库的列名
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty_level"]

# 加载训练集和测试集
train_path=r"F:\Jupyter\kaggle\data\NSL-KDD\KDDTrain+.txt"
test_path=r"F:\Jupyter\kaggle\data\NSL-KDD\KDDTest+.txt"
df = pd.read_csv(train_path, header=None, names = col_names)
df_test = pd.read_csv(test_path, header=None, names = col_names)

df.drop('difficulty_level',inplace=True,axis=1)
df_test.drop('difficulty_level',inplace=True,axis=1)
#数据集的shape
print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)

Dimensions of the Training set: (125973, 42)
Dimensions of the Test set: (22544, 42)


In [58]:
df.drop('flag', axis=1, inplace=True)
df.drop('protocol_type', axis=1, inplace=True)
df.drop('service', axis=1, inplace=True)
# test data

df_test.drop('flag', axis=1, inplace=True)
df_test.drop('protocol_type', axis=1, inplace=True)
df_test.drop('service', axis=1, inplace=True)
print(df.shape)
print(df_test.shape)

(125973, 39)
(22544, 39)


In [59]:
labeldf=df['label']
labeldf_test=df_test['label']

#把.洗掉
labeldf = labeldf.str.rstrip('.')
labeldf_test = labeldf_test.str.rstrip('.')
newlabeldf=labeldf.replace({'normal':0,
                            'neptune':1,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 
                             'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2,
                             'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'worm': 3,
                             'httptunnel': 4,'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({'normal' : 0,
                             'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 
                             'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2,
                             'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'worm': 3,
                             'httptunnel': 4,'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})

df['label'] = newlabeldf
df_test['label'] = newlabeldf_test
# print(newdf['label'].tail())

In [60]:
# 将数据集分割为特征（X）和目标变量（Y）
# assign X as a dataframe of feautures and Y as a series of outcome variables
X = df.drop('label',axis=1)
Y = df.label

X_test = df_test.drop('label',axis=1)
Y_test = df_test.label

In [61]:
colNames=list(X)
colNames_test=list(X_test)

In [63]:
from sklearn.preprocessing import MinMaxScaler

# 创建 MinMaxScaler 对象
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()

# 对训练集特征进行缩放
X_scaled_train = scaler1.fit_transform(X)

# 对测试集特征进行缩放（使用训练集的缩放参数）
X_scaled_test = scaler2.fit_transform(X_test)


In [64]:
from sklearn.naive_bayes import MultinomialNB
start_time = time.time()

# all features
clf = MultinomialNB()
clf.fit(X, Y)
end_time = time.time()
# 计算执行时间
training_time = end_time - start_time
print(f"训练时间: {training_time:.2f} 秒")


训练时间: 0.06 秒


In [65]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
# 使用 cross_val_predict 进行交叉验证预测
y_pred = cross_val_predict(clf, X_test, Y_test, cv=10)

# 混淆矩阵
pd.crosstab(Y_test, y_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])

Predicted attacks,0,1,2,3,4
Actual attacks,,,,,
0,6899,482,1962,204,164
1,0,1715,5219,521,3
2,73,11,2333,0,4
3,663,68,1556,235,232
4,50,13,111,7,19


In [66]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_auc_score,f1_score
print(classification_report(Y_test, y_pred, digits=3))


             precision    recall  f1-score   support

          0      0.898     0.710     0.793      9711
          1      0.749     0.230     0.352      7458
          2      0.209     0.964     0.343      2421
          3      0.243     0.085     0.126      2754
          4      0.045     0.095     0.061       200

avg / total      0.687     0.497     0.511     22544



In [67]:
from sklearn.metrics import confusion_matrix

# 计算混淆矩阵
conf_matrix = confusion_matrix(Y_test, y_pred)

# 提取各分类的真正例、假正例、真负例、假负例
true_positives = np.diag(conf_matrix)
false_positives = np.sum(conf_matrix, axis=0) - true_positives
false_negatives = np.sum(conf_matrix, axis=1) - true_positives
true_negatives = np.sum(conf_matrix) - (true_positives + false_positives + false_negatives)

# 计算各分类的准确率
class_accuracies = true_positives / (true_positives + false_negatives)

# 将整体准确率加入到列表中
overall_accuracy = np.sum(true_positives) / np.sum(conf_matrix)

# 创建包含准确率的数据帧
index = [str(i) for i in range(len(class_accuracies))] + ['Overall Accuracy']
df_ac = pd.DataFrame({'Accuracy': np.append(class_accuracies, overall_accuracy)}, index=index)

# 打印数据帧
print(df_ac)

                  Accuracy
0                 0.710431
1                 0.229954
2                 0.963651
3                 0.085330
4                 0.095000
Overall Accuracy  0.496851
